In [1]:
import os, subprocess
import pandas as pd


# Finn's Laptop
dicombasefolder = '/mnt/d/Finn/KPUM_NODDI/DICOM_KPUM_NODDI'
# KPUM Workstation
#dicombasefolder = '/mnt/e/Finn/KPUM_NODDI/DICOM_KPUM_NODDI'

# sourcedata base folder to go within dicom base folder
sourcedatabasefolder = 'sourcedata'

# tsv-files to keep track of in dicombasefolder
subjecttrackerpath = os.path.join(dicombasefolder, sourcedatabasefolder)
subjecttrackerfile = 'Subject_Tracker_for_dicomdir2sourcedata.tsv'

# Read the Subject_Tracker_dicomdir2sourcedata.tsv
subjecttrackertsv = os.path.join(subjecttrackerpath, subjecttrackerfile)
df = pd.read_csv(subjecttrackertsv, sep="\t")
df.fillna('', inplace=True)

# Loop over entries in Subject_Tracker_dicomdir2sourcedata.tsv
for index, row in df.iterrows() :
    dicomdatafolder = row['dicomfolder']
    print(f'Processing {dicomdatafolder}')

    # Parse didcomdatafolder to get variables
    subject, session, rest  = dicomdatafolder.split("_", 2)

    # Define path and create output dicomdir if not present
    outputdicomdir = os.path.join(dicombasefolder, sourcedatabasefolder, f'sub-{subject}', f'ses-{session}')
    if not os.path.exists(outputdicomdir): # then make this directory
        os.makedirs(outputdicomdir)


    # Check if not converted
    if  [ df.loc[df['dicomfolder'] == dicomdatafolder, ['converted']] != 'Yes' ] :
        # Run bash-script
        print("Start re-organising and anonymizing DCMs")
        result = subprocess.run(f"bash ../shell/nifti_dicomdir2sourcedata.sh {dicombasefolder} {dicomdatafolder} {subject} {session} -sourcedata {sourcedatabasefolder}", stdout=subprocess.PIPE, shell=True)
        #print(result.stdout)
        # Update Subject_Tracker_dicomdir2sourcedata.tsv
        df.loc[df['dicomfolder'] == dicomdatafolder, ['converted']] = 'Yes'
        df.loc[df['dicomfolder'] == dicomdatafolder, ['sourcedatafolder']] = os.path.join(sourcedatabasefolder, f'sub-{subject}', f'ses-{session}')
        df.to_csv(os.path.join(dicombasefolder, subjecttrackertsv), sep="\t", index=False)
        print("Done re-organising and anonymizing DCMs")
    else :
        print("Already converted")


NameError: name 'subject' is not defined